In [1]:
!pip install transformers librosa soundfile torch torchvision torchaudio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 53.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 41.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlink

In [2]:
# 导入必要的库
import torch
import librosa
import soundfile as sf
from transformers import AutoFeatureExtractor, AutoModelForAudioClassification

# 定义模型名称
model_name = "ehcalabres/wav2vec2-lg-xlsr-en-speech-emotion-recognition"

# 初始化特征提取器和模型
feature_extractor = AutoFeatureExtractor.from_pretrained(model_name)
model = AutoModelForAudioClassification.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/214 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.28k [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/configuration_utils.py:312: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


model.safetensors:   0%|          | 0.00/1.27G [00:00<?, ?B/s]

Some weights of the model checkpoint at ehcalabres/wav2vec2-lg-xlsr-en-speech-emotion-recognition were not used when initializing Wav2Vec2ForSequenceClassification: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.output.bias', 'classifier.output.weight']
- This IS expected if you are initializing Wav2Vec2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at ehcalabres/wav2vec2-lg-xlsr-en-speech-emotion-recognition and are newly initialized: ['classifier.bias', 'classifier.weight', '

In [3]:
from google.colab import drive
drive.mount('/content/drive/')
audio_file = "/content/drive/MyDrive/testv/10.wav"  # 确保文件路径正确
audio, sample_rate = sf.read(audio_file)

# 检查并重新采样音频
target_sample_rate = 16000
if sample_rate != target_sample_rate:
    print(f"重新采样音频从 {sample_rate} Hz 到 {target_sample_rate} Hz。")
    audio = librosa.resample(audio, orig_sr=sample_rate, target_sr=target_sample_rate)
    sample_rate = target_sample_rate

# 使用特征提取器处理音频
inputs = feature_extractor(audio, return_tensors="pt", sampling_rate=sample_rate)

# 使用模型进行预测
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits

# 获取预测结果
predicted_class = torch.argmax(logits, dim=-1).item()
print(f"预测的情感类别是: {predicted_class}")


Mounted at /content/drive/
重新采样音频从 32000 Hz 到 16000 Hz。
预测的情感类别是: 2


In [4]:
from google.colab import drive
import soundfile as sf
import librosa
import torch
import torch.nn.functional as F

# 挂载 Google Drive
drive.mount('/content/drive/')

# 加载音频文件
audio_file = "/content/drive/MyDrive/testv/10.wav"  # 确保文件路径正确
audio, sample_rate = sf.read(audio_file)

# 检查并重新采样音频
target_sample_rate = 16000
if sample_rate != target_sample_rate:
    print(f"重新采样音频从 {sample_rate} Hz 到 {target_sample_rate} Hz。")
    audio = librosa.resample(audio, orig_sr=sample_rate, target_sr=target_sample_rate)
    sample_rate = target_sample_rate

# 使用特征提取器处理音频
inputs = feature_extractor(audio, return_tensors="pt", sampling_rate=sample_rate)

# 使用模型进行预测
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits

# 应用 softmax 函数以获得归一化概率分布
probabilities = F.softmax(logits, dim=-1)

# 打印概率分布
print("概率分布:", probabilities)

# 获取预测结果（可选）
predicted_class = torch.argmax(logits, dim=-1).item()
print(f"预测的情感类别是: {predicted_class}")

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
重新采样音频从 32000 Hz 到 16000 Hz。
概率分布: tensor([[0.1196, 0.1318, 0.1331, 0.1300, 0.1223, 0.1267, 0.1199, 0.1165]])
预测的情感类别是: 2
